# ORTHOGONAL POLYNOMIAL DENSITY ESTIMATION
## Preliminaries
### Cange working directory

In [ ]:
cd PyCharm

### Imports

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sbn
import pandas as pd
from uuid import uuid4
from random import getrandbits

from lpde.geometry import WidthOf, Window, PointAt, BoundingBox, Mapper
from lpde.estimate import DensityEstimate
from lpde.estimate.helpers import Event, Degree, Scalings, Coefficients, Action

### Notebook settings

In [3]:
%matplotlib notebook
sbn.set_style('white')

legend_params = {'loc': 'best',
                 'frameon': True, 
                 'fancybox': True, 
                 'framealpha': 0.7}

In [9]:
action = Action.ADD
point = PointAt(1, 1)
uuid = uuid4()

In [ ]:
event = Event(uuid, action, point)

In [ ]:
point = event.location

In [ ]:
point.location

In [5]:
degree = Degree(10, 10)

In [ ]:
scale = Scalings(degree)

In [ ]:
coeffs = Coefficients(degree)

In [ ]:
coeffs.mat

In [ ]:
from collections import namedtuple

In [ ]:
DegreeBase = namedtuple('Degree', ['k_max', 'l_max'])

In [ ]:
test = DegreeBase(1, 2)

In [ ]:
degree._replace(k_max=3)

In [ ]:
class Degree(DegreeBase):
    def __new__(cls, k_max, l_max):
        
        self = super().__new__(cls, k_max, l_max)
        

In [6]:
legendre_width = WidthOf(1.8)

center = PointAt(0, 0)
window = Window(4, 4)
bounds = BoundingBox(center, window)

mapper = Mapper(bounds, legendre_width)

density = DensityEstimate(degree, mapper)

In [7]:
density._c

array([ 1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.])

In [10]:
point = PointAt(0.5, 0.5)
event = Event(uuid4(), action, point)

In [11]:
density.at(event.location)

0.25

In [12]:
density.update_with(event)

ValueError: Length of values does not match length of index

In [ ]:
density._phi

In [ ]:
def new_event():
    location = np.random.uniform(low=-2, high=2, size=2)
    point = PointAt(*location)
    return Event(uuid4(), True, point)

def event():
    add = bool(getrandbits(1))
    if add:
        location = np.random.uniform(low=-2, high=2, size=2)
        point = PointAt(*location)
        return Event(uuid4(), True, point)
    column = density._phi.sample(1, axis=1).columns.values[0]
    return Event(column, False)

In [ ]:
%%time
for i in range(1000):
    density.update_with(new_event())

In [ ]:
%%time
for i in range(1000):
    density.update_with(event())
print(density._phi.shape)

In [ ]:
%%time
for i in range(1000):
    density.at(new_event().location)

__Timings__

144 ms pro additive update

200 ms pro random update

0.5 ms pro ausrechnen

In [ ]:
x_grid = np.linspace(-0.9, 0.90, 50)
y_grid = np.linspace(-0.9, 0.90, 50)
x_grid, y_grid = np.meshgrid(x_grid, y_grid)

In [ ]:
#for i in range(10):
#    density.update_with(new_event())

p_hat = density._on(x_grid.ravel(), y_grid.ravel())

fig, ax = plt.subplots()
ax.set(xlabel=r'$x$', ylabel=r'$y$')
contour = ax.contourf(x_grid, y_grid, p_hat, 9, cmap='inferno')
cbar = plt.colorbar(contour, ax=ax)
fig.tight_layout()

In [ ]:
density._c.dot(density._c)

In [ ]:
4.2/100

In [ ]:
(3*60+27)/1000

In [ ]:
bool(getrandbits(1))

In [ ]:
density._phi.sample(1, axis=1).columns.values[0]